In [ ]:
# sampling methodology template script 
# 20230913
# carverd@colostate.edu

In [10]:
# libraries 
import geemap
import geopandas
import pandas
import ee

In [12]:
# read in inputdatasets 
## index and results  
# df = pandas.read_csv("../data/samplingPlan/samplingReferenceSheet_4mile.csv")
## spatial reference grid
grid = geopandas.read_file("../data/griddedFeatures/Nebraska_Fishnet.gpkg")
### this will be remove once we get more 
grid['id'] = range(1, len(grid.index)+1)
grid = grid[['id','geometry']]

In [15]:
# !!User input required!!
## define aoi by id
aoiID = 200

,id,geometry
0,1,"MULTIPOLYGON (((-104.05373 40.00010, -103.7638..."
1,2,"MULTIPOLYGON (((-103.76387 40.00010, -103.4740..."
2,3,"MULTIPOLYGON (((-103.47402 40.00010, -103.1841..."
3,4,"MULTIPOLYGON (((-103.18416 40.00010, -102.8943..."
4,5,"MULTIPOLYGON (((-102.89431 40.00010, -102.6044..."
...,...,...
336,337,"MULTIPOLYGON (((-96.51750 42.89865, -96.22764 ..."
337,338,"MULTIPOLYGON (((-96.22764 42.89865, -95.93779 ..."
338,339,"MULTIPOLYGON (((-95.93779 42.89865, -95.64793 ..."
339,340,"MULTIPOLYGON (((-95.64793 42.89865, -95.35808 ..."


In [35]:
# Generaete map objects based on the define reference grid 
## Polygon
### as geopandas class 
aoi_pandas = grid[grid.id == aoiID]
aoi_pandas
### as ee object
aoi_ee = geemap.geopandas_to_ee(aoi_pandas)
# aoi_ee
## points --- I don't think this is going to be an interactable object on the map... so we'll just sample presence and absence locations. 
# falsePoints = aoi_pandas.sample_points(size=100).explode()
# falsePoints = geopandas.GeoDataFrame(geometry=geopandas.GeoSeries(falsePoints))
# falsePoints
# falsePoints_ee = geemap.gdf_to_ee(falsePoints)
# falsePoints_ee

In [36]:
# compile naip imagery 
# 2010, 2016, 2020 for sample time periods 
naip2020 = geemap.get_annual_NAIP(2020).filterBounds(aoi_ee).mosaic()
naip2015 = geemap.get_annual_NAIP(2016).filterBounds(aoi_ee).mosaic()
naip2010 = geemap.get_annual_NAIP(2010).filterBounds(aoi_ee).mosaic()

In [37]:
# compile Naip visualizations 
visParamsFalse = {"bands":['N', 'R', 'G']};
visParamsTrue = {"bands":['R', 'G', 'B']};



In [38]:
# generate the initial map object 
Map = geemap.Map()
Map.add_basemap("SATELLITE")
Map.centerObject(ee_object = aoi_ee, zoom = 10)

Map

Map(center=[41.88414123713827, -100.14068581900119], controls=(WidgetControl(options=['position', 'transparent…

In [160]:
# I'm not sure if we want all three naip images on at once or each map getting it's own. 
### 2020 tree sampling 
Map2020 = Map 
Map2020.addLayer(naip2020, visParamsTrue, name = "rgb2020")
Map2020.addLayer(naip2020, visParamsFalse, name = "false2020")
# testing the 2015 layer
Map2020.addLayer(naip2015, visParamsTrue, name = "rgb2015")

Map2020

Map(bottom=779987.0, center=[41.88414123713827, -100.14068581900119], controls=(WidgetControl(options=['positi…

In [156]:
# extract objects 
presence2020 = ee.FeatureCollection(Map.draw_features)
coords = presence2020.geometry()

### ok lots to do here. Probably want to export the feature as a object incase the users exits. the map features are not save on reload. 
# grab the coordinates and assign them to the correct row and column based on the date 
# write out the csv to save progesss

## for subsequent sampling periods. Re


,ID,presence,absence,exported,expectedSampler,sampledBy,DateSampled


In [161]:
Map2015 = Map 
Map2015.addLayer(naip2015, visParamsTrue, name = "rgb2015")
Map2015.addLayer(naip2015, visParamsFalse, name = "false2015")
# testing adding the previous selection to the 2015 naip. --- is this easier then resampling 
Map2015.addLayer(presence2020, name = "2020Presence")

### idea, button that change map zoom from to all points. 

### absence --- generate the random points within gee so that they are editable features. 

Map2015

Map(bottom=12473332.0, center=[41.88414123713827, -100.14068581900119], controls=(WidgetControl(options=['posi…